In [155]:
import pandas as pd
class Individual_diets:
    "Clase que implementa el individuo en el problema de dietas"

    def __init__(self, chromosome,limits):
        self.chromosome = chromosome[:]  #  chromosome =dieta
        self.fitness = -1  #  fitness = -1
        self.limits=limits

        
    def search_limit(self, chromosome):
        for (i,j) in self.chromosome:
             for (food,l) in self.limits:
                    if food==i:
                        return l
        return 0
    
    def crossover(self, other):

        chromosome = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for (i,j) in self.chromosome:
            for (i1,j1) in other.chromosome:
                if(i==i1):
                    chromosome.append((i,(j+j1)/2))
        return Individual_diets(chromosome,self.limits)

    def mutate(self):
        "Cambia aleatoriamente la cantidad de porciones de un alimento en el cromosoma"
        mutated_ind = Individual_diets(self.chromosome[:],self.limits)
        index = random.randint(0, len(mutated_ind.chromosome)-1)
        newPos = random.randint(0, self.search_limit(mutated_ind.chromosome[index]))
        mutated_ind.chromosome[index] = newPos
        return mutated_ind


In [197]:
import random

def genetic_algorithm(population, fitness_fn, ngen=100, pmut=0.1,cal=1350):
    "Algoritmo Genetico "
    
    popsize = len(population)
    evaluate_population(population, fitness_fn,cal)  # evalua la poblacion inicial
    ibest = sorted(range(len(population)), key=lambda i: population[i].fitness)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    
    for g in range(ngen):   # Por cada generacion
        
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 
        
        ## Crea la poblacion descendencia cruzando las parejas del mating pool 
        offspring_population = []
        for i in range(len(mating_pool)): 
            offspring_population.append( mating_pool[i][0].crossover(mating_pool[i][1]) ) # cruzamiento 
          
        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                    offspring_population[i] = offspring_population[i].mutate()   # mutacion de una posicion

        
        ## Evalua la poblacion descendencia creada
        evaluate_population(offspring_population, fitness_fn,cal)  # evalua la poblacion inicial
        
        ## Selecciona popsize individuos para la sgte. generación de la union de la pob. actual y  pob. descendencia
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {},best_cromosoma = {}".format(g, population[ibest[0]].fitness,population[ibest[0]].chromosome))
    
    return population[ibest[0]], bestfitness  

In [198]:
def evaluate_population(population, fitness_fn,cal):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    popsize = len(population)
    for i in range(popsize):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            population[i].fitness = fitness_fn(population[i].chromosome,cal)

In [207]:
def fitness_diet(chromosome,cal):
    """Retorna el fitness de un cromosoma en el problema de dietas """
    data=pd.read_csv('data_Alimentos.csv',delimiter=',')

    for (i,j) in chromosome:
        count=0
        fit=0
        for f in data[data.columns[1]]:
            if f==i:
                fit+= (data.loc[count][3])*j
            count+=1
    fitness= cal-fit;
    return fitness

In [208]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

In [209]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

In [210]:

foodlist=[('apio',2),('fresa',1),('manzana',0),('arroz',1),('aji',0),('lechuga',2),('res',1),('papa',1),('cebolla',0),('leche',0)]



In [211]:
from itertools import combinations, permutations,product
def generate_population(foodlist):
    alimentos=[]
    for (a,b)in foodlist:
        if (b==0):
            alimento=[]
            for i in range(3):
                alimento.append((a,i))
        if (b==1):
            alimento=[]
            for i in range(4,7):
                alimento.append((a,i))
        if (b==2):
            alimento=[]
            for i in range(4,7):
                alimento.append((a,i))
        alimentos.append(alimento)

    productcartesian=product(*alimentos)
    solutions=[]
    for i in productcartesian:
        solutions.append(Individual_diets(i,foodlist))
    return solutions

In [212]:
population=generate_population(foodlist)

In [213]:
population[7].chromosome

(('apio', 4),
 ('fresa', 4),
 ('manzana', 0),
 ('arroz', 4),
 ('aji', 0),
 ('lechuga', 4),
 ('res', 4),
 ('papa', 4),
 ('cebolla', 2),
 ('leche', 1))

In [222]:
best_ind, bestfitness = genetic_algorithm(population, fitness_diet, 2, 0.5,1000)
plt.plot(bestfitness)

Poblacion inicial, best_fitness = 874


KeyboardInterrupt: 

In [219]:
for (i,j) in population[8].chromosome:
    print(i,j)

apio 4
fresa 4
manzana 0
arroz 4
aji 0
lechuga 4
res 4
papa 4
cebolla 2
leche 2


In [216]:
a= population[7].crossover(population[8])

In [217]:
a.chromosome

[('apio', 4.0),
 ('fresa', 4.0),
 ('manzana', 0.0),
 ('arroz', 4.0),
 ('aji', 0.0),
 ('lechuga', 4.0),
 ('res', 4.0),
 ('papa', 4.0),
 ('cebolla', 2.0),
 ('leche', 1.5)]

In [220]:
fitness_diet(population[8].chromosome,1350)

1224